In [1]:
import warnings
import logging
import os
import glob
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Embedding
import re
import statistics

In [15]:
!pip install xlrd==1.1.0

     |████████████████████████████████| 108 kB 6.2 MB/s eta 0:00:01
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.0.0
    Uninstalling xlrd-1.0.0:
      Successfully uninstalled xlrd-1.0.0


In [2]:
warnings.simplefilter('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL
logging.getLogger('tensorflow').setLevel(logging.FATAL)

In [3]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
#EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler, Callback
from tensorflow.keras.losses import BinaryCrossentropy
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import precision_recall_curve
from tensorflow.keras.layers import Add, Dense, LayerNormalization, Masking, GlobalAveragePooling1D, Conv1D, Dropout, MultiHeadAttention, Layer
from tensorflow.keras.callbacks import LearningRateScheduler, Callback, ModelCheckpoint
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.metrics import Recall, Precision , AUC
#from tensorflow_addons.metrics import F1Score
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras import Model
import numpy as np
#from sklearn.metrics import auc, roc_curve
#import matplotlib.pyplot as plt


In [7]:
def preprocessing(mode = 'train'):
    new_len = 256
    in_chan = 3
    fall = os.path.join(os.getcwd() , f'dataset_Alex/{mode}/Fall/***.xlsx')
    adl = os.path.join(os.getcwd(), f'dataset_Alex/{mode}/ADL/***.xlsx')
    fall_files = glob.glob(fall)
    adl_files = glob.glob(adl)

    all_file_path = fall_files + adl_files
    samples = len(all_file_path)
    dataset = np.zeros((1,new_len, in_chan))
    
    if len(all_file_path) == 0:
        raise Exception('Data files not found')
 
    trials_count = {}
    fall_pattern = re.compile("Fall")
    trials = []
    labels = []
    length = []
    #count = 0

    for file_path in all_file_path:
        label = None
        if fall_pattern.search(file_path):
            label = 1
        else:
            label = 0

        #checking if the excel has 2 sheets or not
        if len (pd.ExcelFile(file_path).sheet_names) == 2:
                df = pd.read_excel(file_path, sheet_name=-1)
                df = df.iloc[:, :6]
                null_col = df[df.isnull().any(axis = 1)].index.to_list()
                
                if len(null_col) % 10 != 0  :
                    print(f'{file_path} trimmed file contains {len(null_col)} of null rows')
                    continue
                    
                    #raise Exception(f'{file_path} trimmed file contains {len(null_col)} of null rows')
                #calculating how many null segments we have 
                null_col = df[df.isnull().any(axis = 1)].index.to_list()
                null_seg = len(null_col)//10
                

                trial_start_lst = null_col[9::10]
                trial_end_lst = null_col[10::10]
                
                for i in range(len(null_col)//10 + 1):
                    #trials_count[label] = trials_count.get(label , 0) + 1
                    trial = None
                    if i == 0 :

                        trial = df.iloc[0:null_col[1]-1, 3:6]
                    elif i == null_seg :
                        trial = df.iloc[trial_start_lst[-1]+1:, 3:6]
                    else: 
                        trial_end = trial_end_lst[i-1]
                        trial_start = trial_start_lst[i-1] + 1
                        trial = df.iloc[trial_start : trial_end-1 , 3:6]
                        trial.dropna(inplace = True)
                    
                    #trial = tf.convert_to_tensor(trial.values, dtype = tf.float32)
                    act_len = trial.shape[0]
                    channel = trial.shape[1]
                    x = np.linspace(1, act_len, num=new_len)
                    xp = np.linspace(1, act_len, num= act_len)
                    interpolated_data = np.zeros((1,new_len,channel))
                    trial = trial.to_numpy()
                    if trial.shape[0] == 0:
                        continue
                        
                    else:
                        for idx in range(channel):
                            yp = trial[:, idx]
                            axis_interpolated = np.interp(x, xp, yp)
                            interpolated_data[0,:, idx] = axis_interpolated
                        dataset = np.concatenate((dataset,interpolated_data), axis = 0)
                        labels.append(label)
        
        else:
            raise Exception(f'{file_path} doesnt have trimmed data')
        
    return tf.convert_to_tensor(dataset[1:, :, :]), tf.convert_to_tensor(labels)


In [8]:
def sliding_window_preprocessing(mode = 'train'):
    window_length = 50
    step_size = 25
    in_chan = 3
    #finding the path to all fall and adl data
    fall = os.path.join(os.getcwd() , f'dataset_Alex/{mode}/Fall/***.xlsx')
    adl = os.path.join(os.getcwd(), f'dataset_Alex/{mode}/ADL/***.xlsx')
    fall_files = glob.glob(fall)
    adl_files = glob.glob(adl)

    all_file_path = fall_files + adl_files
    samples = len(all_file_path)
    
    #initiating the dataset array
    dataset = np.zeros((1,new_len, in_chan))
    
    if len(all_file_path) == 0:
        raise Exception('Data files not found')
 
    trials_count = {}
    fall_pattern = re.compile("Fall")
    trials = []
    labels = []
    length = []
    #count = 0

    for file_path in all_file_path:
        label = None
        if fall_pattern.search(file_path):
            label = 1
        else:
            label = 0
            step_size *= 2
        
            

        #checking if the excel has 2 sheets or not
        if len (pd.ExcelFile(file_path).sheet_names) == 2:
                df = pd.read_excel(file_path, sheet_name=-1)
                df = df.iloc[:, :6]
                null_col = df[df.isnull().any(axis = 1)].index.to_list()
                
                if len(null_col) % 10 != 0  :
                    raise Exception(f'{file_path} trimmed file contains {len(null_col)} of null rows')
                #calculating how many null segments we have 
                null_col = df[df.isnull().any(axis = 1)].index.to_list()
                null_seg = len(null_col)//10
                

                trial_start_lst = null_col[9::10]
                trial_end_lst = null_col[10::10]
                
                for i in range(len(null_col)//10 + 1):
                    #trials_count[label] = trials_count.get(label , 0) + 1
                    trial = None
                    if i == 0 :

                        trial = df.iloc[0:null_col[1]-1, 3:6]
                    elif i == null_seg :
                        trial = df.iloc[trial_start_lst[-1]+1:, 3:6]
                    else: 
                        trial_end = trial_end_lst[i-1]
                        trial_start = trial_start_lst[i-1] + 1
                        trial = df.iloc[trial_start : trial_end-1 , 3:6]
                        trial.dropna(inplace = True)
                    
                    #trial = tf.convert_to_tensor(trial.values, dtype = tf.float32)
                    act_len = trial.shape[0]
                    channel = trial.shape[1]
                    x = np.linspace(1, act_len, num=new_len)
                    xp = np.linspace(1, act_len, num= act_len)
                    interpolated_data = np.zeros((1,new_len,channel))
                    trial = trial.to_numpy()
                    if trial.shape[0] == 0:
                        continue
                        
                    else:
                        for idx in range(channel):
                            yp = trial[:, idx]
                            axis_interpolated = np.interp(x, xp, yp)
                            interpolated_data[0,:, idx] = axis_interpolated
                        dataset = np.concatenate((dataset,interpolated_data), axis = 0)
                        labels.append(label)
        
        else:
            raise Exception(f'{file_path} doesnt have trimmed data')
        
    return tf.convert_to_tensor(dataset[1:, :, :]), tf.convert_to_tensor(labels)

In [9]:
#saving the dataset to np file
dataset , labels = preprocessing(mode = "train")
#np.savez('alex_val', trials= dataset , labels = labels)

InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory

In [6]:
class F1_Score(tf.keras.metrics.Metric):

    def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.f1 = self.add_weight(name='f1', initializer='zeros')
        self.precision_fn = Precision(thresholds=0.5)
        self.recall_fn = Recall(thresholds=0.5)

    def update_state(self, y_true, y_pred, sample_weight=None):
        p = self.precision_fn(y_true, y_pred)
        r = self.recall_fn(y_true, y_pred)
        # since f1 is a variable, we use assign
        self.f1.assign(2 * ((p * r) / (p + r + 1e-6)))

    def result(self):
        return self.f1

    def reset_states(self):
        # we also need to reset the state of the precision and recall objects
        self.precision_fn.reset_states()
        self.recall_fn.reset_states()
        self.f1.assign(0)

## Lr Scheduler

In [7]:

import math

def cosine_schedule(base_lr, total_steps, warmup_steps ):
    def step_fn(epoch):
        lr = base_lr 
        progress = (epoch - warmup_steps) / float(total_steps -  warmup_steps)

        progress = tf.clip_by_value(progress, 0.0, 1.0)

        lr = lr * 0.5 * (1.0 + tf.cos(math.pi * progress))
        
        if warmup_steps:
            lr = lr * tf.minimum(1.0 , epoch/warmup_steps)
        
        return lr
    

    return step_fn

## Positional Embedding

In [8]:

class PositionalEmbedding(Layer):
    def __init__(self, units,dropout_rate,  **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)

        self.units = units
        self.conv_1 = Conv1D(filters  = units, kernel_size = 1)
        self.projection = Dense(units, kernel_initializer=TruncatedNormal(stddev=0.02))

        self.dropout = Dropout(rate=dropout_rate)

    def build(self, input_shape):
        super(PositionalEmbedding, self).build(input_shape)

        self.position = self.add_weight(
            name="position",
            shape=(1, input_shape[1], self.units),
            initializer=TruncatedNormal(stddev=0.02),
            trainable=True,
        )

    def call(self, inputs, training):
        x = self.projection(inputs)
        # x = self.conv_1(inputs)
        x = x + self.position
        return self.dropout(x, training=training)

## Encoder

In [9]:
class Encoder(Layer):
    def __init__(
        self, embed_dim, mlp_dim, num_heads, dropout_rate,
        attention_dropout_rate, **kwargs
    ):
        super(Encoder, self).__init__(**kwargs)

        self.mha = MultiHeadAttention(
            num_heads = num_heads,
            key_dim = embed_dim,
            dropout = attention_dropout_rate, 
            kernel_initializer = TruncatedNormal(stddev = 0.02)
        )

        self. dense_0 = Dense(
            units = mlp_dim, 
            activation = "gelu", 
            kernel_initializer = TruncatedNormal(stddev = 0.02)
        )

        self.dense_1 = Dense(
            units = embed_dim, 
            kernel_initializer = TruncatedNormal(stddev = 0.02)
        )

        self.conv_0 = Conv1D(filters = 4 , kernel_size = 1, activation = 'relu')
        self.conv_1 = Conv1D(filters  = embed_dim, kernel_size = 1)

        self.dropout_0 = Dropout(rate = dropout_rate)
        self.dropout_1 = Dropout(rate = dropout_rate)

        self.norm_0 = LayerNormalization(epsilon = 1e-6)
        self.norm_1 = LayerNormalization(epsilon = 1e-6)

        self.add_0 = Add()
        self.add_1 = Add()
    
    def call(self, inputs, training):


        x = self.norm_0(inputs)
        x = self.mha(
            query = x, 
            value = x, 
            key = x,
            training = training
        ) #[batch_size, sequence_length, embed_dim][8, 500, 3]
        x = self.dropout_0(x, training= training)
        x = self.add_0([x, inputs])

        #MLP block 
        y = self.norm_1(x)
        y = self.dense_0(y) #[batch_size , sequence_length, neurons]
        y = self.dropout_1(y, training)
        y = self.dense_1(y)#[batch_size , sequence_lenght, neurons]
        y = self.dropout_1(y, training)
        

        return self.add_1([x, y])

## Transformer

In [10]:
class Transformer(Model):
    def __init__(
        self,
        num_layers,
        embed_dim,
        mlp_dim,
        num_heads,
        num_classes,
        dropout_rate,
        attention_dropout_rate,
        **kwargs
    ):
        super(Transformer, self).__init__(**kwargs)

        # Input (normalization of RAW measurements)
        self.input_norm = Normalization()
        

        # Input
        self.pos_embs = PositionalEmbedding(embed_dim, dropout_rate)

        # Encoder
        self.e_layers = [
            Encoder(embed_dim, mlp_dim, num_heads, dropout_rate, attention_dropout_rate)
            for _ in range(num_layers)
        ]

        # Output
        self.norm = LayerNormalization(epsilon=1e-5)
        self.pool = GlobalAveragePooling1D(data_format = 'channels_first')
        self.dense_0 = Dense(mlp_dim, activation = 'relu')
        self.final_layer = Dense(1, kernel_initializer="zeros", activation = 'sigmoid')

    def call(self, inputs, training = True):
        #expanded_input = tf.cast(tf.tile(tf.expand_dims(inputs, axis=-2), [1, 1, 500,1]), tf.float32)
        #self.masking_layer.build(expanded_input.shape)
        #mask = self.masking_layer.compute_mask(expanded_input)
        x = self.input_norm(inputs) 
        x = self.pos_embs(x, training=training)
        for layer in self.e_layers:
            x = layer(x, training=training)
        x = self.norm(x)
        x = self.pool(x)
        x = self.dense_0(x)

        x = self.final_layer(x)
        return x

## Train Function 

In [11]:
    
    #model.input_norm.adapt(X_train, batch_size=config['batch_size'])
    config = {
      'epochs': 50,
      'num_layers':  2,
      'embed_layer_size': 3,
      'global_clipnorm' : 3.0,
      'fc_layer_size': 256,
      'num_heads': 4,
      'dropout': 0.1,
      'attention_dropout': 0.1,
      'optimizer': 'adam',
      'amsgrad': False,
      'label_smoothing': 0.1,
      'learning_rate': 1e-3,
      #'weight_decay': {
      #    'values': [2.5e-4, 1e-4, 5e-5, 1e-5]
      'warmup_steps': 5,
      'batch_size': 8}
  

In [12]:
  def create_model(data):


    # config = wandb.config
    
    # Generate new model
    model = Transformer(
      num_layers=config['num_layers'],
      embed_dim=config['embed_layer_size'],
      mlp_dim=config['fc_layer_size'],
      num_heads=config['num_heads'],
      num_classes=2,
      dropout_rate=config['dropout'],
      attention_dropout_rate=config['attention_dropout'],
    )

    

    # adapt on training dataset - must be before model.compile !!!
    model.input_norm.adapt(data)

    # Select optimizer
    if config['optimizer'] == "adam":
      optim = Adam(
          global_clipnorm=config['global_clipnorm'],
          amsgrad=config['amsgrad'],
      )
    # elif config.optimizer == "adamw":
    #   optim = AdamW(
    #       weight_decay=config.weight_decay,
    #       amsgrad=config.amsgrad,
    #       global_clipnorm=config.global_clipnorm,
    #       exclude_from_weight_decay=["position"]
    #   )
    else:
      raise ValueError("The used optimizer is not in list of available")

    model.compile(
      loss= BinaryCrossentropy(label_smoothing=config['label_smoothing']),
      optimizer=optim,
      metrics=[Recall(), Precision() , AUC(), F1_Score()],
    )
    
    return model

In [13]:
def train():
    model = create_model(data = X_train)
    checkpoint_filepath = os.path.join(os.getcwd(), 'tmp/weights.ckpt')
    model_checkpoint = ModelCheckpoint(filepath = checkpoint_filepath, 
                                      save_weights_only = True, 
                                      monitor = 'val_f1_score', 
                                      mode = 'max', 
                                      save_best_only = True, 
                                      verbose = True)

    log_dir = "logs/"  # Specify the directory where TensorBoard logs will be saved
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    # print(model.input_norm.variables)

    # Train model
    history = model.fit(
      X_train,
      y_train,
      batch_size=config['batch_size'],
      epochs=config['epochs'],
      validation_data=(X_val, y_val),
      shuffle = True,
      callbacks=[
        LearningRateScheduler(cosine_schedule(base_lr=config['learning_rate'], total_steps=config['epochs'], warmup_steps=config['warmup_steps'])),
        #EarlyStopping(monitor="loss", mode='min', min_delta=0.001, patience=5),
        model_checkpoint, tensorboard_callback
      ],
      verbose=1
    )
    return model

In [4]:
    model = None
    train_dataset_path = os.path.join(os.getcwd(), 'alex_train.npz')
    val_dataset_path = os.path.join(os.getcwd(), 'alex_val.npz')
    #loading train dataset
    train_data = np.load(train_dataset_path)
    X_train = train_data['trials']
    y_train = train_data['labels']
    
    #loading val dataset 
    val_data = np.load(val_dataset_path)
    X_val = val_data['trials']
    y_val = val_data['labels']
    


In [15]:
with tf.device('/gpu:0'):
      model = train()

Epoch 1/50


KeyboardInterrupt: 

In [6]:
# test data
test_dataset_path = os.path.join(os.getcwd(), 'alex_test.npz')
test_data = np.load(test_dataset_path)

X_test = test_data['trials']
y_test = test_data['labels']

In [11]:
X_test.shape

(55, 256, 3)

In [21]:
dataset = tf.data.Dataset.from_tensor_slices(X_test).batch(1)

In [14]:
dataset = dataset.window(1, shift = 10, drop_remainder=True)

In [20]:
for elements in dataset:
    print(elements.shape)

(8, 256, 3)
(8, 256, 3)
(8, 256, 3)
(8, 256, 3)
(8, 256, 3)
(8, 256, 3)
(7, 256, 3)


### Loading the Best Weights

In [17]:
model = create_model(data=X_train)
model.evaluate(X_test, y_test, batch_size=8, steps=X_test.shape[0]/8)

#loading best weights and evaluating
weight_path = 'tmp/weights.ckpt'
model.load_weights(weight_path)
model.evaluate(X_test, y_test, batch_size=8, steps=X_test.shape[0]/8)

6/6 [==============================] - 0s 18ms/step - loss: 0.6633 - recall_2: 0.8000 - precision_2: 0.5333 - auc_1: 0.7093 - f1_score: 0.6400


[0.6632546782493591,
 0.800000011920929,
 0.5333333611488342,
 0.7092856764793396,
 0.639999508857727]

In [61]:
model.summary()

Model: "transformer_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization_2 (Normalizati multiple                  7         
_________________________________________________________________
positional_embedding_2 (Posi multiple                  780       
_________________________________________________________________
encoder_4 (Encoder)          multiple                  1990      
_________________________________________________________________
encoder_5 (Encoder)          multiple                  1990      
_________________________________________________________________
layer_normalization_14 (Laye multiple                  6         
_________________________________________________________________
global_average_pooling1d_2 ( multiple                  0         
_________________________________________________________________
dense_19 (Dense)             multiple                

In [62]:
graph = tf.compat.v1.get_default_graph()

In [65]:
model.layers

## Ploting ROC Curve

In [22]:
y_pred = model.predict(X_test).ravel()
fp_pred, tp_pred, threshold_pred = roc_curve(y_test, y_pred)


2/2 [==============================] - 1s 336ms/step


In [35]:
y_pred 

array([0.8889392 , 0.16474193, 0.99482226, 0.9363375 , 0.95260257,
       0.7315567 , 0.96716213, 0.71308637, 0.9427625 , 0.7331089 ,
       0.53011787, 0.98499864, 0.99883133, 0.91073406, 0.82420546,
       0.8478556 , 0.9957637 , 0.98465466, 0.99582255, 0.99457103,
       0.76983017, 0.7252614 , 0.48596117, 0.41546848, 0.19717704,
       0.0412966 , 0.21211296, 0.6689365 , 0.37978172, 0.4929512 ,
       0.9272632 , 0.86242265, 0.92437226, 0.95877945, 0.7537029 ,
       0.9250761 , 0.11383366, 0.07067526, 0.10001323, 0.33242407,
       0.05488832, 0.07872478, 0.1397683 , 0.06928135, 0.2589604 ,
       0.03677565, 0.04694097, 0.03518001, 0.06578131, 0.04774711,
       0.06822002, 0.04714935, 0.03687645, 0.4182752 , 0.13995308],
      dtype=float32)

In [34]:
arry.astype(int)

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [29]:
y_test

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [ ]:
auc = auc(fp_pred, tp_pred)
#pr = precision_recall_curve(y_test, y_pred)

In [82]:
#plt.figure()
#plt.plot([0,1], [0, 1], 'k--')
#plt.plot(fp_pred, tp_pred, label = 'SmartFall (area = {:.3f})'.format(auc))
#plt.xlabel('False Positive Rate')
#plt.ylabel('True Positive Rate')
#plt.title('PR Curve')
#plt.show()



from sklearn.metrics import precision_recall_curve
import plotly.express as px

precision, recall, thresholds = precision_recall_curve(y_test, y_pred)
    #fpr, tpr, _ = roc_curve(y_real, y_pred)
    
plt.plot(recall, precision, marker='.')
plt.xlabel('Recall')
plt.ylabel('Precision')
title = 'Precision-Recall Curve\nauc: {:.4f}'.format(auc)
plt.title(title)
#title=f"{title} (AUC={auc(false_positive_rate, true_positive_rate):.4f})",
plt.fill_between(recall, precision, alpha=0.2)
plt.xlim([0.0, 1])
plt.xticks([i * 0.5 for i in range(3)])
x = np.linspace(0, 1, 100)
y = -x + 1
plt.plot(x, y, color='r', linestyle='--', label='y = -x')



plt.show()




ModuleNotFoundError: No module named 'plotly'

### Creating Tflite model

In [ ]:
tf.keras.models.save_model(model, 'transformer', overwrite=True, save_format="tf")

In [33]:
cd transformer/

/home/bgu9/KD_Multimodal/transformer


In [34]:
ls

assets/  saved_model.pb  variables/


In [20]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)


In [21]:
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops. <-- Add this line
]
converter.target_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter.allow_custom_ops=True
converter.experimental_new_converter =True
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

In [22]:
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

In [23]:
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

In [24]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [25]:
data = X_test[1, :, :]

In [26]:
data = data[np.newaxis, :]

In [ ]:
# Set input tensor to the interpreter
interpreter.set_tensor(input_details[0]['index'], data.astype(np.float32))

# Run inference
interpreter.invoke()

# Get the output tensor and post-process the results (example)
output_data = interpreter.get_tensor(output_details[0]['index'])
print("Inference result:", output_data)

In [51]:
model.summary()

Model: "transformer"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization multiple                  7         
_________________________________________________________________


ValueError: You tried to call `count_params` on masking, but the layer isn't built. You can build it manually via: `masking.build(batch_input_shape)`.